In [5]:
from os import path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from sklearn import linear_model
from geopy.distance import great_circle

In [6]:
nypd = pd.read_csv(path.join('data', 'NYPD_Motor_Vehicle_Collisions.csv'), parse_dates=[['DATE', 'TIME']],
                   dtype={'ZIP CODE': str})

/anaconda3/envs/modin-pandas/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
print(nypd.columns)

Index(['DATE_TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE', 'LONGITUDE', 'LOCATION',
       'ON STREET NAME', 'CROSS STREET NAME', 'OFF STREET NAME',
       'NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED',
       'NUMBER OF PEDESTRIANS INJURED', 'NUMBER OF PEDESTRIANS KILLED',
       'NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED',
       'NUMBER OF MOTORIST INJURED', 'NUMBER OF MOTORIST KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
       'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
       'CONTRIBUTING FACTOR VEHICLE 5', 'UNIQUE KEY', 'VEHICLE TYPE CODE 1',
       'VEHICLE TYPE CODE 2', 'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4',
       'VEHICLE TYPE CODE 5'],
      dtype='object')


#### Q: What is the total number of persons injured in the dataset (up to December 31, 2018)?
#### A: 367837. (Killed is not considered as injured.)

In [12]:
nypd_trunc = nypd.query('DATE_TIME <= "2018-12-31"')
print(nypd_trunc['NUMBER OF PERSONS INJURED'].sum())

367837.0


#### Q: What proportion of collisions in 2016 resulted in injury or death of a cyclist?
#### A: 0.0216547426

In [13]:
nypd_index = nypd.set_index('DATE_TIME')
print(((nypd_index['2016']['NUMBER OF CYCLIST INJURED'] > 0) | (nypd_index['2016']['NUMBER OF CYCLIST KILLED'] > 0)).mean())

0.021654742632339373


#### Q: What proportion of all collisions in 2016 occured in Brooklyn? Only consider entries with a non-null value for BOROUGH.

In [14]:
nypd_2016 = nypd_index['2016'].copy()
print((nypd_2016[~nypd_2016['BOROUGH'].isna()]['BOROUGH'] == 'BROOKLYN').mean())

0.30961778079314234


#### Q: Obtain the number of vehicles involved in each collision in 2016. Group the collisions by zip code and compute the sum of all vehicles involved in collisions in each zip code, then report the maximum of these values.

In [15]:
nypd_2016['num_of_vehicles'] = nypd_2016.filter(like='CONTRIBUTING').notna().sum(axis=1)
print(nypd_2016.groupby('ZIP CODE').count()['num_of_vehicles'].max())

2698


#### Q: For each borough, compute the number of accidents per capita involving alcohol in 2017. Report the highest rate among the 5 boroughs. 

In [16]:
population = pd.DataFrame({'BOROUGH': ['BRONX', 'MANHATTAN', 'QUEENS', 'BROOKLYN', 'STATEN ISLAND'],
                           'population': [1471160, 1664727, 2358582, 2648771, 479458]})
nypd_2017 = nypd_index['2017']
num_acc = nypd_2017[(nypd_2017.filter(like='CONTRIBUTING') == 'Alcohol Involvement').any(axis=1)]\
         .groupby('BOROUGH')['CONTRIBUTING FACTOR VEHICLE 1'].count().reset_index()
num_acc = num_acc.merge(population, on='BOROUGH').set_index('BOROUGH')
print((num_acc['CONTRIBUTING FACTOR VEHICLE 1'] / num_acc['population']).max())

0.00022727521556223623


#### Q: Consider the total number of collisions each year from 2013-2018. Is there an apparent trend? Fit a linear regression for the number of collisions per year and report its slope.

In [17]:
nypd_13_18 = nypd_index['2013':'2018']
data = nypd_13_18.resample('Y').count()['UNIQUE KEY'].reset_index()
x = np.arange(6).reshape(-1, 1)
y = data['UNIQUE KEY']
lr = linear_model.LinearRegression()
fit = lr.fit(x, y)
print(fit.coef_[0])

6447.914285714288


#### Q: Compute the rate of multi car collisions as the proportion of the number of collisions involving 3 or more cars to the total number of collisions for each month of 2017. Calculate the chi-square test statistic for testing whether a collision is more likely to involve 3 or more cars in January than in May.

In [18]:
nypd_2017 = nypd_index['2017']

In [19]:
multi = nypd_2017[nypd_2017.filter(like='CONTRIBUTING').notna().sum(axis=1) >= 3]
colli = nypd_2017[~(nypd_2017.filter(like='CONTRIBUTING').notna().sum(axis=1) >= 3)]
num_colli_mult = multi.resample('M').count()['UNIQUE KEY']
num_collision = colli.resample('M').count()['UNIQUE KEY']

In [20]:
collision = pd.concat([num_collision, num_colli_mult], axis=1)
collision.columns=['coll_few', 'coll_mult']

In [67]:
print(collision)
observed = pd.concat([collision['2017-01'], collision['2017-05']], axis=0).values
chisq, p, dof, expected = stats.chi2_contingency(observed)
print(chisq)

            coll_few  coll_mult
DATE_TIME                      
2017-01-31     16416       1135
2017-02-28     14870        965
2017-03-31     18240       1096
2017-04-30     16731       1098
2017-05-31     19704       1308
2017-06-30     20027       1342
2017-07-31     18319       1274
2017-08-31     17892       1242
2017-09-30     18308       1296
2017-10-31     19081       1277
2017-11-30     18443       1218
2017-12-31     18446       1269
0.9023834507906645


#### Q: We can use collision locations to estimate the areas of the zip code regions. Represent each as an ellipse with semi-axes given by a single standard deviation of the longitude and latitude. For collisions in 2017, estimate the number of collisions per square kilometer of each zip code region. Considering zipcodes with at least 1000 collisions, report the greatest value for collisions per square kilometer. Note: Some entries may have invalid or incorrect (latitude, longitude) coordinates. Drop any values that are invalid or seem unreasonable for New York City.

In [139]:
nypd_2017 = nypd_index['2017']

lat_min = 40.
lat_max = 41.
lon_min = -74.27
lon_max = -73.6
clean_2017 = nypd_2017[(nypd_2017['LATITUDE'].between(lat_min, lat_max)) & (nypd_2017['LONGITUDE'].between(lon_min, lon_max))]

counts = clean_2017.groupby('ZIP CODE').count()
considered = counts[counts['UNIQUE KEY'] >= 1000]['UNIQUE KEY'] # index: float64, count: int64

data = clean_2017[clean_2017['ZIP CODE'].astype('float64').isin(considered.index)]

ellipse = pd.concat([data.groupby('ZIP CODE').agg({'LATITUDE': 'mean', 'LONGITUDE': 'mean'}),
           data.groupby('ZIP CODE').agg({'LATITUDE': 'std', 'LONGITUDE': 'std'})], axis=1)
ellipse.columns = ['lat_mean', 'lon_mean', 'lat_std', 'lon_std']

def area_in_kilo(row):
    a = great_circle((row['lat_mean'], row['lon_mean']), (row['lat_mean'], row['lon_mean'] + row['lon_std'])).kilometers
    b = great_circle((row['lat_mean'], row['lon_mean']), (row['lat_mean'] + row['lat_std'], row['lon_mean'])).kilometers
    return np.pi * a * b

ellipse['area_kilo'] = ellipse.apply(area_in_kilo, axis=1)

combined = pd.concat([considered, ellipse], axis=1)

print((combined['UNIQUE KEY'] / combined['area_kilo']).max())

5129.4471146398255
